In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as scs
import scipy.linalg as spla

import koma.oma
import koma.plot
import koma.clustering

In [2]:
parquet_file = r'./data/data_synced.parquet'
df = pd.read_parquet(parquet_file)

df_filtered = df.drop(['T1E_x', 'T1E_y', 'T1E_z'], axis=1)
df_filtered = df_filtered.drop(df.index[0:16*9*3600+30000])
df_filtered = df_filtered.drop(df.index[-1])
df_filtered = df_filtered.drop(df.index[16*10*3600:-1])

acceleration_data = df_filtered.to_numpy()

fs = 16
t = np.arange(0, len(acceleration_data))

acceleration_data = scs.detrend(acceleration_data, axis=0)

In [3]:
nperseg = 1000
zp = 8
nfft =  nperseg * zp

Sxy = np.array([[scs.csd(ch1, ch2, fs, nperseg=nperseg, nfft=nfft)[1] 
               for ch1 in acceleration_data.T] for ch2 in acceleration_data.T])

In [4]:
i = 200
order = np.arange(2, 62, 2)

stabcrit = {'freq':0.05, 'damping':0.2, 'mac':0.1}
s = 2
valid_range = {'freq':[0, np.inf], 'damping':[0, 0.3], 'mpc':[0.4, 1]}

fs = 16

In [5]:
lambd, phi, orders = koma.oma.covssi(acceleration_data, fs, i, order, return_flat=True)
lambd_stab, phi_stab, orders_stab, ix_stab = koma.oma.find_stable_poles(lambd, phi, orders, s, stabcrit=stabcrit, valid_range=valid_range, indicator='freq', use_legacy=False)

*** Covariance-driven SSI algorithm for OMA ***
> Establishing Hankel/Toeplitz matrices
  >> Correlation estimation
  >> Matrix stacking
> Establishing weighting matrices
  >> Weighting requested: NONE
> Computing SVD
> Computing state matrix for each order to establish modes
> Computation completed


c:\Users\marku\anaconda3\Lib\site-packages\koma\modal.py:272: RuntimeWarning: divide by zero encountered in scalar divide
  eta = (Syy-Sxx)/(2*Sxy)
c:\Users\marku\anaconda3\Lib\site-packages\koma\modal.py:274: RuntimeWarning: invalid value encountered in scalar multiply
  lambda1 = (Sxx+Syy)/2 + Sxy*np.sqrt(eta**2+1)
c:\Users\marku\anaconda3\Lib\site-packages\koma\modal.py:275: RuntimeWarning: invalid value encountered in scalar multiply
  lambda2 = (Sxx+Syy)/2 - Sxy*np.sqrt(eta**2+1)


In [ ]:
fig = koma.plot.stabplot(lambd_stab, orders_stab, psd_freq=f, psd_y=S, frequency_unit='hz', freq_range=[0, 8], damped_freq=False)

fig

In [6]:
pole_clusterer = koma.clustering.PoleClusterer(lambd_stab, phi_stab, orders_stab, min_cluster_size=7, min_samples=7,
                                               scaling={'mac':0.4, 'omega_n':0.8}, normalize_distances=False)

prob_threshold = 1.0

pole_clusterer.cluster()

outputs = pole_clusterer.postprocess(prob_threshold=prob_threshold, normalize_and_maxreal=True)
lambd_used, phi_used, orders_stab_used, group_ixs, all_single_ix, probs = outputs

xi_auto, omega_n_auto, phi_auto, order_auto, ixs_auto, probs_auto = koma.clustering.group_clusters(*outputs)

xi_mean = np.array([np.mean(xi_i) for xi_i in xi_auto])
fn_mean = np.array([np.mean(om_i) for om_i in omega_n_auto])/2/np.pi
xi_std = np.array([np.std(xi_i) for xi_i in xi_auto])
fn_std = np.array([np.std(om_i) for om_i in omega_n_auto])/2/np.pi

grouped_ixs = koma.clustering.group_array(all_single_ix, group_ixs)
grouped_phis = koma.clustering.group_array(phi_used, group_ixs, axis=1)
phi_mean = np.vstack([np.mean(phi_i, axis=1) for phi_i in grouped_phis]).T

In [7]:
fn_mean

array([0.29987612, 0.38392749, 0.38392754, 0.61968127, 0.61968231,
       0.82024045, 1.0875541 , 1.22047103, 1.36068959, 1.61746534,
       1.93119273, 2.11135723, 2.44782797, 2.56052181, 2.74577981,
       2.99192379, 3.32956465, 3.55722866, 4.53539585, 5.66780205,
       5.66796279, 6.74360612])

In [8]:
fn_mean_2 = np.delete(fn_mean, (2, 4, -2))
fn_mean_2

array([0.29987612, 0.38392749, 0.61968127, 0.82024045, 1.0875541 ,
       1.22047103, 1.36068959, 1.61746534, 1.93119273, 2.11135723,
       2.44782797, 2.56052181, 2.74577981, 2.99192379, 3.32956465,
       3.55722866, 4.53539585, 5.66780205, 6.74360612])

In [9]:
phi_mean_2 = np.delete(phi_mean, [2, 4, -2], axis=1)

phi_mean_2.shape

(57, 19)

In [10]:
indexed_freq_ssi = np.zeros((len(fn_mean_2), 2))
indexed_freq_ssi[:,0] = np.array([i for i in range(1, len(fn_mean_2) + 1)])
indexed_freq_ssi[:,1] = fn_mean_2
indexed_freq_ssi

array([[ 1.        ,  0.29987612],
       [ 2.        ,  0.38392749],
       [ 3.        ,  0.61968127],
       [ 4.        ,  0.82024045],
       [ 5.        ,  1.0875541 ],
       [ 6.        ,  1.22047103],
       [ 7.        ,  1.36068959],
       [ 8.        ,  1.61746534],
       [ 9.        ,  1.93119273],
       [10.        ,  2.11135723],
       [11.        ,  2.44782797],
       [12.        ,  2.56052181],
       [13.        ,  2.74577981],
       [14.        ,  2.99192379],
       [15.        ,  3.32956465],
       [16.        ,  3.55722866],
       [17.        ,  4.53539585],
       [18.        ,  5.66780205],
       [19.        ,  6.74360612]])

In [11]:
t1w = phi_mean_2[54:, :]
phi_mean_2 = np.append(phi_mean_2, t1w, 0)

ind = np.array([
    16.1, 16.2, 16.3,
    15.1, 15.2, 15.3,
    9.1,  9.2,  9.3,
    3.1,  3.2,  3.3,
    10.1, 10.2, 10.3,
    4.1,  4.2,  4.3,
    11.1, 11.2, 11.3,
    5.1,  5.2,  5.3,
    12.1, 12.2, 12.3,
    6.1,  6.2,  6.3,
    13.1, 13.2, 13.3,
    7.1,  7.2,  7.3,
    14.1, 14.2, 14.3,
    8.1,  8.2,  8.3,
    19.1, 19.2, 19.3,
    17.1, 17.2, 17.3,
    20.1, 20.2, 20.3,
    18.1, 18.2, 18.3,
    1.1,  1.2,  1.3,
    2.1,  2.2,  2.3,
])

indexed_modes_ssi = np.zeros((60, 20))

indexed_modes_ssi[:,0] = ind
indexed_modes_ssi[:,1:] = phi_mean_2

indexed_modes_ssi = indexed_modes_ssi[indexed_modes_ssi[:, 0].argsort()]

indexed_modes_ssi

C:\Users\marku\AppData\Local\Temp\ipykernel_20920\3931707156.py:30: ComplexWarning: Casting complex values to real discards the imaginary part
  indexed_modes_ssi[:,1:] = phi_mean_2


array([[ 1.10000000e+00,  2.73125099e-02,  9.48690991e-02, ...,
        -9.53374159e-02,  1.49451141e-02, -1.87680702e-04],
       [ 1.20000000e+00,  2.21052150e-01, -7.89744585e-03, ...,
         6.79996482e-03,  1.54828669e-02,  1.16586763e-04],
       [ 1.30000000e+00,  1.57142006e-02,  4.64418624e-03, ...,
         3.24955759e-02, -2.68554265e-02, -1.60613931e-03],
       ...,
       [ 2.01000000e+01, -7.00896032e-02,  3.86483425e-03, ...,
         3.16718361e-02, -8.81981628e-02,  1.01232525e-03],
       [ 2.02000000e+01,  5.14424204e-01, -2.27331091e-02, ...,
        -4.04690533e-02,  9.10580717e-02, -7.72056335e-03],
       [ 2.03000000e+01,  1.49008476e-01, -2.79219488e-02, ...,
         2.57769595e-01, -4.04250689e-01, -1.13097991e-02]])

In [12]:
ordered_modes_ssi = indexed_modes_ssi[:, 1:]
ordered_modes_ssi.shape

(60, 19)

In [13]:
np.save("results/grenland_bridge_frequencies.npy", indexed_freq_ssi)
np.savetxt("results/grenland_bridge_frequencies.txt", indexed_freq_ssi)
np.save("results/grenland_bridge_modes.npy", ordered_modes_ssi)
np.savetxt("results/grenland_bridge_modes.txt", ordered_modes_ssi)